# Movielens 영화 추천 실습

추천시스템의 MNIST라고 부를만한 Movielens 데이터를 이용해서  
영화 추천 시스템을 만들어보자.

유저들이 영화의 평점을 매긴 데이터이다.

별점 데이터는 대표적인 explicit 데이터이지만, 별점을 시청횟수로  
해석해서 implicit 데이터로 간주하고 테스트해보자.

또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외한다.

## 0. 모듈 import

In [20]:
import os
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

## 1. 데이터 준비와 전처리

rating.dat 안에 Movielens 데이터를 불러오자.

In [21]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


불러온 데이터를 확인해보자.

In [22]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1000209 non-null  int64
 1   movie_id   1000209 non-null  int64
 2   rating     1000209 non-null  int64
 3   timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


총 1000209개의 데이터가 있고 결측치는 없다.  
유저아이디, 영화아이디, 점수, 시간의 4개의 colume이 존재한다.

위에서 말했듯이 별점을 시청횟수로 간주하고 3점이하는 버린다.

In [23]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [24]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'view_count'}, inplace=True)
ratings.describe()

,user_id,movie_id,view_count,timestamp
count,836478.000000,836478.000000,836478.000000,8.364780e+05
mean,3033.120626,1849.099114,3.958293,9.721628e+08
std,1729.255651,1091.870094,0.762280,1.206216e+07
min,1.000000,1.000000,3.000000,9.567039e+08
25%,1531.000000,1029.000000,3.000000,9.652795e+08
50%,3080.000000,1747.000000,4.000000,9.728388e+08
75%,4485.000000,2763.000000,5.000000,9.752064e+08
max,6040.000000,3952.000000,5.000000,1.046455e+09


필터링된 데이터는 836478개의 데이터가 있다.  

데이터를 보면 일단 timestamp는 딱히 필요없어보인다.  
그리고 다른 문제는 유저아이디와 영화아이디의 최소값이 1이다.  
즉, 번호를 1번부터 세는데, 나중에 matrix에 매핑할경우,  
인덱스는 0부터이므로 이 부분도 수정이 필요할 것 같다.

그 외에도 영화가 단순히 아이디로만 나와있기 때문에 아이디와 영화타이틀을  
연결시켜줄 데이터를 불러와서 합쳐주자.

In [25]:
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [26]:
# movies와 ratings를 합쳐준다.
data= pd.merge(movies, ratings)
cols = [ 'user_id','title', 'view_count', 'movie_id']
data = data[cols]
data.head()

,user_id,title,view_count,movie_id
0,1,Toy Story (1995),5,1
1,6,Toy Story (1995),4,1
2,8,Toy Story (1995),4,1
3,9,Toy Story (1995),5,1
4,10,Toy Story (1995),5,1


In [27]:
# 고유한 타이틀을 찾아내는 코드
title_unique = data['title'].unique()

title_to_idx = {v:k for k,v in enumerate(title_unique)}

# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_id_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_movie_id_data) == len(data):
    print('movie_id column indexing OK!!')
    data['movie_id'] = temp_movie_id_data
else:
    print('movie_id column indexing Fail!!')

data.head()

movie_id column indexing OK!!


,user_id,title,view_count,movie_id
0,1,Toy Story (1995),5,0
1,6,Toy Story (1995),4,0
2,8,Toy Story (1995),4,0
3,9,Toy Story (1995),5,0
4,10,Toy Story (1995),5,0


## 2. 분석

In [28]:
data.describe()

,user_id,view_count,movie_id
count,836478.000000,836478.000000,836478.000000
mean,3033.120626,3.958293,1680.387078
std,1729.255651,0.762280,992.075521
min,1.000000,3.000000,0.000000
25%,1531.000000,3.000000,942.000000
50%,3080.000000,4.000000,1559.000000
75%,4485.000000,5.000000,2508.000000
max,6040.000000,5.000000,3627.000000


In [29]:
print(f"유니크한 영화 개수 : {data['movie_id'].nunique()}")
print(f"유니크한 사용자 수 : {data['user_id'].nunique()}")

유니크한 영화 개수 : 3628
유니크한 사용자 수 : 6039


In [30]:
print("가장 인기 있는 영화 20개")
data.groupby('title')['user_id'].count().sort_values(ascending=False).head(20)

가장 인기 있는 영화 20개


title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## 3. 내가 선호하는 영화를 5가지 골라서 추가

In [31]:
# 내가 좋아하시는 영화의 데이터로 바꿔서 추가한다.
my_favorite = ['Titanic (1997)', 'Sixth Sense, The (1999)', 'Shawshank Redemption, The (1994)', 'Alien (1979)', 'Die Hard (1988)']
my_favorite_id = [title_to_idx[x] for x in my_favorite]

my_playlist = pd.DataFrame({'user_id': ['kwansu']*5, 'title': my_favorite, 'movie_id':my_favorite_id, 'view_count':[5]*5})

if not data.isin({'user_id':['kwansu']})['user_id'].any():  
    data = data.append(my_playlist)   
    
data.tail(10)

,user_id,title,view_count,movie_id
836473,5682,"Contender, The (2000)",3,3627
836474,5812,"Contender, The (2000)",4,3627
836475,5831,"Contender, The (2000)",3,3627
836476,5837,"Contender, The (2000)",4,3627
836477,5998,"Contender, The (2000)",4,3627
0,kwansu,Titanic (1997),5,1540
1,kwansu,"Sixth Sense, The (1999)",5,2507
2,kwansu,"Shawshank Redemption, The (1994)",5,305
3,kwansu,Alien (1979),5,1098
4,kwansu,Die Hard (1988),5,949


kwansu라는 user_id가 아닌데다 matrix로 만들기 위해 id를 인덱스 순서로 넣어주자.

In [32]:
# 고유한 유저, 타이틀을 찾아내는 코드
user_unique = data['user_id'].unique()

# 유저, 타이틀 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')
    
data.tail(10)

user_id column indexing OK!!


,user_id,title,view_count,movie_id
836473,2696,"Contender, The (2000)",3,3627
836474,2161,"Contender, The (2000)",4,3627
836475,1931,"Contender, The (2000)",3,3627
836476,2162,"Contender, The (2000)",4,3627
836477,3134,"Contender, The (2000)",4,3627
0,6039,Titanic (1997),5,1540
1,6039,"Sixth Sense, The (1999)",5,2507
2,6039,"Shawshank Redemption, The (1994)",5,305
3,6039,Alien (1979),5,1098
4,6039,Die Hard (1988),5,949


## 4. CSR matrix 만들기

In [33]:
num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()

csr_data = csr_matrix((data.view_count, (data.user_id, data.movie_id)), shape=(num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [34]:
print(f"유니크한 영화 개수 : {data['movie_id'].nunique()}")
print(f"유니크한 사용자 수 : {data['user_id'].nunique()}")
print(f'CSR matrix의 크기 : {csr_data.shape}')

유니크한 영화 개수 : 3628
유니크한 사용자 수 : 6040
CSR matrix의 크기 : (6040, 3628)


사용자에 나 자신 'kwansu'를 포함한 6040명과  
총 영화 개수 3628개의 CSR 행렬이 잘 만들어진 것을 확인 할 수 있다.

## 5. 모델 구성 및 훈련

In [45]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=50, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

als_model.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

## 6. 훈련된 모델이 예측한 나의 선호도 파악

200 벡터로 분해된 벡터들을 확인해보자.

In [46]:
kwansu = user_to_idx['kwansu']
kwansu_vector = als_model.user_factors[kwansu]

kwansu_vector

array([ 1.25568673e-01, -2.74167233e-03, -7.10283443e-02,  4.44298446e-01,
       -5.58168054e-01, -2.47826487e-01,  1.66025937e-01, -1.25213295e-01,
        2.14092046e-01,  4.11083519e-01, -4.11800057e-01,  3.97916317e-01,
       -7.09772766e-01, -4.35470104e-01, -5.85334785e-02, -1.23645803e-02,
        1.86502516e-01, -2.70894676e-01, -4.03234735e-02,  2.40782738e-01,
       -3.71534862e-02,  9.12126452e-02, -3.33768725e-01,  5.15889674e-02,
        5.16377874e-02, -1.26121730e-01,  1.74363032e-01,  6.51619971e-01,
       -2.40067348e-01, -1.96008638e-01,  3.74800116e-02, -1.11757323e-01,
        6.24691322e-02, -1.31393299e-01, -2.40660876e-01, -8.69611651e-02,
       -2.60068327e-01,  1.13219202e-01, -4.66706902e-01, -7.55535588e-02,
        2.61957943e-03,  3.65824133e-01, -3.54913846e-02,  2.45839193e-01,
       -6.44224882e-01,  2.42593274e-01, -6.80310205e-02,  1.24267712e-01,
        1.84557274e-01, -9.27338004e-02,  2.15958655e-01,  3.75406981e-01,
       -2.26452515e-01, -

In [47]:
shawshank = title_to_idx['Shawshank Redemption, The (1994)']
shawshank_vector = als_model.item_factors[shawshank]

shawshank_vector.shape

(200,)

내 자신이 좋아한다고 학습시킨 영화에 대한 선호도

In [48]:
np.dot(kwansu_vector, shawshank_vector)
print(f'영화 쇼생크 탈출에 대한 선호도 : {np.dot(kwansu_vector, shawshank_vector)}')

영화 쇼생크 탈출에 대한 선호도 : 0.6779145002365112


In [49]:
silence = title_to_idx['Silence of the Lambs, The (1991)']
silence_vector = als_model.item_factors[silence]
print(f'영화 양들에 침묵에 대한 선호도 : {np.dot(kwansu_vector, silence_vector)}')

영화 양들에 침묵에 대한 선호도 : 0.4809550344944


## 7. 좋아하는 영화와 비슷한 영화를 추천

In [50]:
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [51]:
get_similar_title('Shawshank Redemption, The (1994)')

['Shawshank Redemption, The (1994)',
 'Silence of the Lambs, The (1991)',
 'Pulp Fiction (1994)',
 'Good Will Hunting (1997)',
 'GoodFellas (1990)',
 'Fargo (1996)',
 "Schindler's List (1993)",
 'Dead Man Walking (1995)',
 'Brother Minister: The Assassination of Malcolm X (1994)',
 'Son of Dracula (1943)']

In [52]:
get_similar_title('Alien (1979)')

['Alien (1979)',
 'Aliens (1986)',
 'Terminator, The (1984)',
 'Jaws (1975)',
 'Invasion of the Body Snatchers (1956)',
 'Alien³ (1992)',
 'Blade Runner (1982)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Back Stage (2000)',
 'Alien: Resurrection (1997)']

위의 결과들을 보면 나름 비슷한 유형의 영화들을 잘 추천해주는 것 같다.

## 8. 내가 가장 좋아할 만한 영화들을 추천

In [53]:
user = user_to_idx['kwansu']

# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)

print("내가 좋아할 만한 영화 추천 목록 : ")
for i in title_recommended:
    print(f'영화이름 : {idx_to_title[i[0]]:50s} 선호도 : {i[1]:f}')

내가 좋아할 만한 영화 추천 목록 : 
영화이름 : Silence of the Lambs, The (1991)                   선호도 : 0.480955
영화이름 : Terminator, The (1984)                             선호도 : 0.461898
영화이름 : Aliens (1986)                                      선호도 : 0.438358
영화이름 : Jaws (1975)                                        선호도 : 0.299320
영화이름 : Raiders of the Lost Ark (1981)                     선호도 : 0.251855
영화이름 : Pulp Fiction (1994)                                선호도 : 0.228851
영화이름 : Schindler's List (1993)                            선호도 : 0.217845
영화이름 : Jerry Maguire (1996)                               선호도 : 0.211932
영화이름 : Rocky (1976)                                       선호도 : 0.211865
영화이름 : Predator (1987)                                    선호도 : 0.210351
영화이름 : Terminator 2: Judgment Day (1991)                  선호도 : 0.208525
영화이름 : Star Wars: Episode IV - A New Hope (1977)          선호도 : 0.203096
영화이름 : Indiana Jones and the Last Crusade (1989)          선호도 : 0.198687
영화이름 : Fargo (1996)          

결과가 생각보다 내 취향의 영화들이 나왔다.  
예상보다 학습결과가 좋은 것 같다.

그럼 다음으로는 가장 추천하는 영화 터미너이터와 양들의 침묵에 대한 기여도를 확인해보자.

In [54]:
terminator = title_to_idx['Terminator, The (1984)']
explain = als_model.explain(user, csr_data, itemid=terminator)

print('터미네이터에 대한 기여도 :')
[(idx_to_title[i[0]], i[1]) for i in explain[1]]


터미네이터에 대한 기여도 :


[('Die Hard (1988)', 0.2541255047690311),
 ('Alien (1979)', 0.1883153247166049),
 ('Sixth Sense, The (1999)', 0.02122794028406909),
 ('Titanic (1997)', -0.0008528946891768014),
 ('Shawshank Redemption, The (1994)', -0.0037453307478643658)]

In [55]:
silence = title_to_idx['Silence of the Lambs, The (1991)']
explain = als_model.explain(user, csr_data, itemid=silence)

print('양들에 침묵에 대한 기여도 :')
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

양들에 침묵에 대한 기여도 :


[('Shawshank Redemption, The (1994)', 0.2931562916574094),
 ('Sixth Sense, The (1999)', 0.129930780406022),
 ('Die Hard (1988)', 0.0386205746951783),
 ('Titanic (1997)', 0.010007979015329835),
 ('Alien (1979)', 0.006259764898336246)]

터미네이터는 비슷한 액션 장르인 다이하드의 기여도가 가장 높았고,  
반대로 다른 전혀 다른 장르인 식스센스의 기여도는 음수였다.

양들의 침묵은 비슷한 장르인 쇼생크 탈출이 가장 높고,  
스케일 때문인지 타이타닉이 가장 낮은 기여도를 보였다.

기여도가 어느정도 잘 반영된 것 같다.

## 9. 회고

처음 벡터의 크기가 100일 때는 좋아하는 영화에 대해 너무 낮은 선호도가 나와  
300으로 늘렸지만, 다른 영화들에 대한 선호도가 너무 떨어졌다.  
예상으로는 기존 5개의 정보에 너무 오버피팅 된거 같아서 적당히 200개 크기로 잡았다.

특히 이번 프로젝트를 하면서 rating matrix와  Matrix Factorization에 대해서 많이 알게 된 것 같다.  
그러면서 머신러닝을 통한 추천 시스템에 대한 기본적인 지식이 많이 쌓인것 같다.